In [3]:
from openai_client import OpenAIClient
from pydantic import BaseModel
import json

class QueryResponse(BaseModel):
    answer: str
client = OpenAIClient(role = "You talk only as a pirate", reasoning={'effort':'minimal'}, verbosity='low')


In [ ]:


res = client.invoke(query="What is javascript?",
        json_schema=QueryResponse
    )['text']
json.loads(res)


In [ ]:
# use input in notebook to interactively chat with user using chatbot
while True:
    user_input = input("User: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting chat.")
        break
    response = client.invoke(query=user_input, json_schema=QueryResponse)
    print(f"AI: {response.answer}")

In [ ]:
react_prompt = """Answer the following questions as best you can. You have access to the tools provided.

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do. 
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None


Thought: Next step based on the observation. This can also be an additional step that you need to take based on the information.
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None

Thought: Now I know the final answer
FinalAnswer: The final answer to the original input question based on all the evidence so far. 

1. (Thought/Action/Observation/FinalAnswer can be repeated zero or more times)
2. FinalAnswer is None until the end, when it has the final answer to the original input question
3. If the thoughts, actions and observations which have happened so far are sufficient for the final answer, then your next thought must be "Now I know the final answer"
4. If you see an observation as the last step, generate a thought
5. If the thought says "Now I know the final answer", then read the full coversation and action results so far and give the final answer
6. The user starts with the question and you generate the first thought
7. If you notice that some thoughts and actions have already happened after the conversation starts, you should continue from there

Begin!"""

class Thought(BaseModel):
    thought: str

class FinalAnswer(BaseModel):
    final_answer: str
    
max_iter = 3
convo_so_far = ""

user_question = "What are the latest developments in artificial intelligence? Also tell me which topics are part of top AI Research and curriculums?"
for i in range(max_iter):
    thought_client = OpenAIClient(role = "You are an expert agent that can think and use tools to answer user questions")